In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
%matplotlib inline
import glob
import cv2
from skimage import feature
from sklearn.model_selection import train_test_split
from natsort import natsorted

try:
    import xgboost as xgb
except:
    print('XGBoost not loaded.')

/home/mikel/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def data_loader(vehicle_loc='./vehicles/', non_loc='./non-vehicles/', train_pct=0.75):
    veh_files = glob.glob(vehicle_loc + '**/*.png', recursive=True)
    non_files = glob.glob(non_loc + '**/*.png', recursive=True)
    
    print('[DataLoader] Found {} vehicle and {} non-vehicle images. Loading...'.format(len(veh_files), len(non_files)))

    x_train = []
    x_valid = []
    y_train = []
    y_valid = []
    
    # Do train/validation split _properly_
    for target, dataset in enumerate([non_files, veh_files]):
        folders = list(set(['/'.join(f.split('/')[:-1]) for f in dataset])) # Get unique folders
        print(folders)
        for folder in folders:
            # Sort each folder by time series
            folder_files = natsorted([f for f in dataset if folder in f])
            # Split into train and validation
            folder_files_train = folder_files[:int(len(folder_files) * train_pct)]
            folder_files_valid = folder_files[int(len(folder_files) * train_pct):]
            
            print('Loading ({}) {} - {} train {} valid'.format(target, folder, len(folder_files_train), len(folder_files_valid)))
            
            folder_imgs_train = [cv2.imread(f) for f in folder_files_train]
            folder_imgs_valid = [cv2.imread(f) for f in folder_files_valid]
            
            x_train.extend(folder_imgs_train)
            x_valid.extend(folder_imgs_valid)
            y_train.extend([target for _ in folder_files_train])
            y_valid.extend([target for _ in folder_files_valid])
    
    return np.array(x_train), np.array(x_valid), np.array(y_train), np.array(y_valid)

In [ ]:
# Function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def bin_spatial(img, size=(32, 32)):
    features = cv2.resize(img, size).ravel()
    return features

# Extract features from an image
def process_image(img):
    if img.shape != (64, 64, 3):
        img = cv2.resize(img, (64, 64))
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    hog = feature.hog(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), transform_sqrt=True, orientations=8, cells_per_block=(2, 2))
    hist = color_hist(img)
    hist2 = color_hist(hls_img)
    spatial = bin_spatial(img, size=(16, 16))
    spatial2 = bin_spatial(hls_img, size=(16, 16))
    return np.concatenate([hog, hist, hist2, spatial, spatial2])

In [26]:
# Take in an image, and split into many sliding windows
def sliding_window_sampler(img, offset=32, block_size=64):
    data_img = []
    shape = img.shape

    for xo in range(((shape[0] - block_size) // offset)+1):
        for yo in range(((shape[1] - block_size) // offset)+1):  
            mini_img = img[xo*offset:xo*offset+block_size, yo*offset:yo*offset+block_size, :]
            data_img.append(mini_img)
    return np.array(data_img)

from multiprocessing import Pool
pool = Pool(12) # Open 12 multiprocessing threads

# Search over many different search spaces simulatenously
def sliding_window_dual(input_img, clf, gbm, search_spaces, std=None):
    heatmap = np.zeros(input_img.shape[:2])
    heatmap2 = np.zeros(input_img.shape[:2])
    
    # Repeat in each search space
    for top, bottom, left, right, block_size, offset in search_spaces:
        img = input_img[top:bottom, left:right]
        shape = img.shape
        #print(img.shape)
        
        # Get windows
        data_img = sliding_window_sampler(img, offset=offset, block_size=block_size)
        
        # Multithreaded feature generation
        x_img = np.array(pool.map(process_image, data_img))

        # Predict with both models
        p2 = gbm.predict(xgb.DMatrix(x_img))
        p = clf.predict(std.transform(x_img))

        # Loop over the blocks, adding to the heatmap
        i = 0
        for xo in range(((shape[0] - block_size) // offset)+1):
            for yo in range(((shape[1] - block_size) // offset)+1):    
                heatmap[(xo*offset)+top:(xo*offset+block_size)+top, (yo*offset)+left:(yo*offset+block_size)+left] += p[i]
                i += 1
        
        i = 0
        for xo in range(((shape[0] - block_size) // offset)+1):
            for yo in range(((shape[1] - block_size) // offset)+1):    
                heatmap2[(xo*offset)+top:(xo*offset+block_size)+top, (yo*offset)+left:(yo*offset+block_size)+left] += p2[i]
                i += 1
        
    # Return the two heatmaps
    return heatmap, heatmap2

In [11]:
# Train an XGBoost model
def run_xgb(x_train, y_train, x_valid, y_valid):
    # Define parameters for XGBoost
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = ['auc', 'error']
    params['eta'] = 0.1
    params['colsample_bylevel'] = 0.1
    params['subsample'] = 0.8
    params['max_depth'] = 20
    
    # Convert data to C++ DataMatrix format
    d_train = xgb.DMatrix(x_train, y_train)
    d_valid = xgb.DMatrix(x_valid, y_valid)
    
    # Define datasets
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    
    # Train XGBoost with early stopping
    clf = xgb.train(params, d_train, 1000, watchlist, verbose_eval=10, early_stopping_rounds=100)
    return clf

In [9]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

# Train a LinearSVC with standard-scaling
def run_sklearn(x_train, y_train, x_valid, y_valid):
    
    std = StandardScaler()
    x_train = std.fit_transform(x_train)
    x_valid = std.transform(x_valid)
    
    clf = LinearSVC() #SVC(kernel='linear')
    clf.fit(x_train, y_train)
    p_train = clf.predict(x_train)
    p_valid = clf.predict(x_valid)
    
    # Training and validation accuracy
    print((p_train == y_train).mean(), (p_valid == y_valid).mean())
    return clf, std

In [13]:
# Load data
x_train, x_valid, y_train, y_valid = data_loader()
# Create features
x2_train = np.array([process_image(img) for img in x_train])
x2_valid = np.array([process_image(img) for img in x_valid])
# Train linear SVM
clf, std = run_sklearn(x2_train, y_train, x2_valid, y_valid)
# Train XGBoost
gbm = run_xgb(x2_train, y_train, x2_valid, y_valid)

[DataLoader] Found 8792 vehicle and 8968 non-vehicle images. Loading...
['./non-vehicles/non-vehicles/GTI', './non-vehicles/non-vehicles/Extras']
Loading (0) ./non-vehicles/non-vehicles/GTI - 2925 train 975 valid
Loading (0) ./non-vehicles/non-vehicles/Extras - 3801 train 1267 valid
['./vehicles/vehicles/KITTI_extracted', './vehicles/vehicles/GTI_Far', './vehicles/vehicles/GTI_MiddleClose', './vehicles/vehicles/GTI_Right', './vehicles/vehicles/GTI_Left']
Loading (1) ./vehicles/vehicles/KITTI_extracted - 4474 train 1492 valid
Loading (1) ./vehicles/vehicles/GTI_Far - 625 train 209 valid
Loading (1) ./vehicles/vehicles/GTI_MiddleClose - 314 train 105 valid
Loading (1) ./vehicles/vehicles/GTI_Right - 498 train 166 valid
Loading (1) ./vehicles/vehicles/GTI_Left - 681 train 228 valid
1.0 0.977037370554
[0]	train-auc:0.997609	train-error:0.013215	valid-auc:0.980029	valid-error:0.053579
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until va

In [14]:
from skimage import measure

# This function takes in a raw image and car mask, and finds all connected pixels to form bounding boxes
# around the cars
def binary_mask_to_boxes(img, heatmap):
    img = img.copy()
    clusters = measure.label(heatmap)
    for label in np.unique(clusters)[1:]:
        where = np.where(clusters == label)
        xmin, xmax, ymin, ymax = where[1].min(), where[1].max(), where[0].min(), where[0].max()
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 6)
    return img

In [ ]:
last_heatmap = None
last_heatmap2 = None
alpha = 0.2

def run_img(img):
    global last_heatmap, last_heatmap2
    rgb = img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # These are the defined sliding window parameters. The format is as follows:
    # (ymin, ymax, xmin, xmax, window_size, stride)
    search_spaces = [
    (380, 500, 0, 1280, 64, 16),
    (380, 600, 0, 1280, 96, 32),
    (400, 700, 0, 1280, 128, 32),
    (400, 700, 0, 1280, 256, 64)
    ]
    
    # Do a sliding window search to get heatmaps
    heatmap, heatmap2 = sliding_window_dual(img, clf, gbm, search_spaces, std)
    
    # Do a weighted mean with the last frame
    if last_heatmap is not None:
        heatmap = (heatmap * alpha) + (last_heatmap * (1 - alpha))
        heatmap2 = (heatmap2 * alpha) + (last_heatmap2 * (1 - alpha))
    last_heatmap = heatmap
    last_heatmap2 = heatmap2
    
    # Create a mask from the intersection of the two heatmaps
    mask = (heatmap > 3) & (heatmap2 > 0.1)
    
    # Apply bounding boxes to images
    output = binary_mask_to_boxes(rgb, mask)
    plt.imshow(output)
    plt.show()
    return output
    
    
from moviepy.editor import VideoFileClip
white_output = 'out16.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(run_img) #NOTE: this function expects color images!!

%time white_clip.write_videofile(white_output, audio=False)